In [10]:
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install mdatagen


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install --upgrade pip
!pip install virny


  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\zhossai3\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import virny
print(virny.__version__)


0.6.0


In [13]:
#Install using an HTTP link
!pip install git+https://github.com/DataResponsibly/Virny.git@feature/add_datasets_for_benchmark

#Install using an SSH link
!pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/add_datasets_for_benchmark

  Cloning https://github.com/DataResponsibly/Virny.git (to revision feature/add_datasets_for_benchmark) to c:\users\zhossai3\appdata\local\temp\pip-req-build-ryhr1555


  Running command git clone --filter=blob:none --quiet https://github.com/DataResponsibly/Virny.git 'C:\Users\zhossai3\AppData\Local\Temp\pip-req-build-ryhr1555'
  Running command git checkout -q feature/add_datasets_for_benchmark
  error: pathspec 'feature/add_datasets_for_benchmark' did not match any file(s) known to git
  error: subprocess-exited-with-error
  
  × git checkout -q feature/add_datasets_for_benchmark did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
error: subprocess-exited-with-error

× git checkout -q feature/add_datasets_for_benchmark did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


  Cloning ssh://****@github.com/DataResponsibly/Virny.git (to revision feature/add_datasets_for_benchmark) to c:\users\zhossai3\appdata\local\temp\pip-req-build-s043kqlf


  Running command git clone --filter=blob:none --quiet 'ssh://****@github.com/DataResponsibly/Virny.git' 'C:\Users\zhossai3\AppData\Local\Temp\pip-req-build-s043kqlf'
  Host key verification failed.
  fatal: Could not read from remote repository.

  Please make sure you have the correct access rights
  and the repository exists.
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet 'ssh://****@github.com/DataResponsibly/Virny.git' 'C:\Users\zhossai3\AppData\Local\Temp\pip-req-build-s043kqlf' did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet 'ssh://****@github.com/DataResponsibly/Virny.git' 'C:\Users\zhossai3\AppData\Local\Temp\pip-req-build-s043kqlf'

In [14]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [16]:
import pandas as pd
import matplotlib.pyplot as plt

from virny.datasets import GermanCreditDataset
from virny.utils.model_tuning_utils import tune_ML_models
from virny.preprocessing.basic_preprocessing import preprocess_dataset
from virny.utils.data_viz_utils import create_dataset_stats_bar_chart
from virny.utils.custom_initializers import create_models_config_from_tuned_params_df

from pprint import pprint
from datetime import datetime, timezone
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from EDA_utils import get_correlation_with_target, get_correlation_matrix, get_features_by_target_correlation_threshold

In [17]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from geomloss import SamplesLoss

import ot

import os
import pickle as pkl
import copy

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer


import argparse
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
import pandas as pd

torch.set_default_tensor_type('torch.DoubleTensor')

**Data Load**

In [18]:
DATASET_SPLIT_SEED = 100
MODELS_TUNING_SEED = 100
TEST_SET_FRACTION = 0.2
DATASET_NAME = 'German_Credit'

sensitive_attributes_dct = {'sex': 'female', 'age': [i for i in range(19, 26)], 'sex & age': None}
sensitive_attributes = [attr for attr in sensitive_attributes_dct.keys() if '&' not in attr]
data = GermanCreditDataset().full_df
data.head()

,checking-account,duration,credit-history,purpose,credit-amount,savings-account,employment-since,installment-rate,other-debtors,residence-since,...,other-installment,housing,existing-credits,job,number-people-provide-maintenance-for,telephone,foreign-worker,sex,marital-status,class-label
0,<0 DM,6,1,radio/television,1169,no savings account,>=7 years,4,none,4,...,none,own,2,1,1,yes,yes,male,divorced/separated,1
1,0 <= <200 DM,48,2,radio/television,5951,<100 DM,1<= < 4 years,2,none,2,...,none,own,1,1,1,none,yes,female,divorced/separated,0
2,no account,12,1,education,2096,<100 DM,4<= <7 years,2,none,3,...,none,own,1,2,2,none,yes,male,divorced/separated,1
3,<0 DM,42,2,furniture/equipment,7882,<100 DM,4<= <7 years,2,guarantor,4,...,none,for free,1,1,2,none,yes,male,divorced/separated,1
4,<0 DM,24,3,car (new),4870,<100 DM,1<= < 4 years,3,none,4,...,none,for free,2,1,2,none,yes,male,divorced/separated,0


In [19]:
print(type(data))
print(data.info())
print(data.columns)
null_value = data.isnull().sum()
print(null_value)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   checking-account                       1000 non-null   object
 1   duration                               1000 non-null   int64 
 2   credit-history                         1000 non-null   object
 3   purpose                                1000 non-null   object
 4   credit-amount                          1000 non-null   int64 
 5   savings-account                        1000 non-null   object
 6   employment-since                       1000 non-null   object
 7   installment-rate                       1000 non-null   int64 
 8   other-debtors                          1000 non-null   object
 9   residence-since                        1000 non-null   int64 
 10  property                               1000 non

In [20]:
print('Checking account',data['checking-account'].unique())
print('duration',data['duration'].unique())
print('credit-history',data['credit-history'].unique())
print('purpose',data['purpose'].unique())
print('credit-amount',data['credit-amount'].unique())
print('savings-account',data['savings-account'].unique())
print('employment-since',data['employment-since'].unique())
print('installment-rate',data['installment-rate'].unique())
print('other-debtors',data['other-debtors'].unique())
print('residence-since',data['residence-since'].unique())
print('property',data['property'].unique())
print('age',data['age'].unique())
print('other-installment',data['other-installment'].unique())
print('housing',data['housing'].unique())
print('existing-credits',data['existing-credits'].unique())
print('job',data['job'].unique())
print('number-people-provide-maintenance-for',data['number-people-provide-maintenance-for'].unique())
print('telephone',data['telephone'].unique())
print('foreign-worker',data['foreign-worker'].unique())
print('sex',data['sex'].unique())
print('marital-status',data['marital-status'].unique())
print('class-label',data['class-label'].unique())

Checking account ['<0 DM' '0 <= <200 DM' 'no account' '>= 200 DM ']
duration [ 6 48 12 42 24 36 30 15  9 10  7 60 18 45 11 27  8 54 20 14 33 21 16  4
 47 13 22 39 28  5 26 72 40]
credit-history ['1' '2' '3' '4' '5']
purpose ['radio/television' 'education' 'furniture/equipment' 'car (new)'
 'car (used)' 'business' 'domestic appliances' 'repairs' 'others'
 'retraining']
credit-amount [ 1169  5951  2096  7882  4870  9055  2835  6948  3059  5234  1295  4308
  1567  1199  1403  1282  2424  8072 12579  3430  2134  2647  2241  1804
  2069  1374   426   409  2415  6836  1913  4020  5866  1264  1474  4746
  6110  2100  1225   458  2333  1158  6204  6187  6143  1393  2299  1352
  7228  2073  5965  1262  3378  2225   783  6468  9566  1961  6229  1391
  1537  1953 14421  3181  5190  2171  1007  1819  2394  8133   730  1164
  5954  1977  1526  3965  4771  9436  3832  5943  1213  1568  1755  2315
  1412 12612  2249  1108   618  1409   797  3617  1318 15945  2012  2622
  2337  7057  1469  2323   932 

In [21]:
from sklearn.preprocessing import LabelEncoder

# Initialize the encoder
label_encoder = LabelEncoder()

data["checking-account"] = label_encoder.fit_transform(data["checking-account"])
data["credit-history"] = label_encoder.fit_transform(data["credit-history"])
data["purpose"] = label_encoder.fit_transform(data["purpose"])
data["savings-account"] = label_encoder.fit_transform(data["savings-account"])
data["employment-since"] = label_encoder.fit_transform(data["employment-since"])
data["other-debtors"] = label_encoder.fit_transform(data["other-debtors"])
data["property"] = label_encoder.fit_transform(data["property"])
data["other-installment"] = label_encoder.fit_transform(data["other-installment"])
data["job"] = label_encoder.fit_transform(data["job"])
data["telephone"] = label_encoder.fit_transform(data["telephone"])
data["foreign-worker"] = label_encoder.fit_transform(data["foreign-worker"])
data["sex"] = label_encoder.fit_transform(data["sex"])
data["marital-status"] = label_encoder.fit_transform(data["marital-status"])
data["housing"] = label_encoder.fit_transform(data["housing"])


In [22]:
print(data.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype
---  ------                                 --------------  -----
 0   checking-account                       1000 non-null   int64
 1   duration                               1000 non-null   int64
 2   credit-history                         1000 non-null   int64
 3   purpose                                1000 non-null   int64
 4   credit-amount                          1000 non-null   int64
 5   savings-account                        1000 non-null   int64
 6   employment-since                       1000 non-null   int64
 7   installment-rate                       1000 non-null   int64
 8   other-debtors                          1000 non-null   int64
 9   residence-since                        1000 non-null   int64
 10  property                               1000 non-null   int64
 11  age                            

In [23]:
data.head()

,checking-account,duration,credit-history,purpose,credit-amount,savings-account,employment-since,installment-rate,other-debtors,residence-since,...,other-installment,housing,existing-credits,job,number-people-provide-maintenance-for,telephone,foreign-worker,sex,marital-status,class-label
0,1,6,0,7,1169,4,3,4,2,4,...,1,1,2,0,1,1,1,1,0,1
1,0,48,1,7,5951,2,0,2,2,2,...,1,1,1,0,1,0,1,0,0,0
2,3,12,0,4,2096,2,1,2,2,3,...,1,1,1,1,2,0,1,1,0,1
3,1,42,1,5,7882,2,1,2,1,4,...,1,0,1,0,2,0,1,1,0,1
4,1,24,2,1,4870,2,0,3,2,4,...,1,0,2,0,2,0,1,1,0,0


In [24]:
print('Checking account',data['checking-account'].unique())
print('duration',data['duration'].unique())
print('credit-history',data['credit-history'].unique())
print('purpose',data['purpose'].unique())
print('credit-amount',data['credit-amount'].unique())
print('savings-account',data['savings-account'].unique())
print('employment-since',data['employment-since'].unique())
print('installment-rate',data['installment-rate'].unique())
print('other-debtors',data['other-debtors'].unique())
print('residence-since',data['residence-since'].unique())
print('property',data['property'].unique())
print('age',data['age'].unique())
print('other-installment',data['other-installment'].unique())
print('housing',data['housing'].unique())
print('existing-credits',data['existing-credits'].unique())
print('job',data['job'].unique())
print('number-people-provide-maintenance-for',data['number-people-provide-maintenance-for'].unique())
print('telephone',data['telephone'].unique())
print('foreign-worker',data['foreign-worker'].unique())
print('sex',data['sex'].unique())
print('marital-status',data['marital-status'].unique())
print('class-label',data['class-label'].unique())

Checking account [1 0 3 2]
duration [ 6 48 12 42 24 36 30 15  9 10  7 60 18 45 11 27  8 54 20 14 33 21 16  4
 47 13 22 39 28  5 26 72 40]
credit-history [0 1 2 3 4]
purpose [7 4 5 1 2 0 3 8 6 9]
credit-amount [ 1169  5951  2096  7882  4870  9055  2835  6948  3059  5234  1295  4308
  1567  1199  1403  1282  2424  8072 12579  3430  2134  2647  2241  1804
  2069  1374   426   409  2415  6836  1913  4020  5866  1264  1474  4746
  6110  2100  1225   458  2333  1158  6204  6187  6143  1393  2299  1352
  7228  2073  5965  1262  3378  2225   783  6468  9566  1961  6229  1391
  1537  1953 14421  3181  5190  2171  1007  1819  2394  8133   730  1164
  5954  1977  1526  3965  4771  9436  3832  5943  1213  1568  1755  2315
  1412 12612  2249  1108   618  1409   797  3617  1318 15945  2012  2622
  2337  7057  1469  2323   932  1919  2445 11938  6458  6078  7721  1410
  1449   392  6260  7855  1680  3578  7174  2132  4281  2366  1835  3868
  1768   781  1924  2121   701   639  1860  3499  8487  6887 